<a href="https://colab.research.google.com/github/azzbc7819/test_igaimers/blob/develop/v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split


In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':6e-4,
    'BATCH_SIZE':4096,
    'SEED':41
}

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
brand_keyword_cnt = pd.read_csv('/content/drive/MyDrive/open/brand_keyword_cnt.csv')
sales = pd.read_csv('/content/drive/MyDrive/open/sales.csv')

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID', '제품'])

In [ ]:
train_date = train_data.iloc[:,5:]

In [ ]:
train_date.T.dtypes

0        int64
1        int64
2        int64
3        int64
4        int64
         ...  
28889    int64
28890    int64
28891    int64
28892    int64
28893    int64
Length: 28894, dtype: object

In [ ]:
summary_stats = train_date.T[5:].describe()

# 평균(mean)과 표준편차(std)를 추출합니다.
mean_values = summary_stats.loc['mean']
std_values = summary_stats.loc['std']

In [ ]:
train_summary_mean = pd.DataFrame(mean_values.T)
train_summary_std = pd.DataFrame(std_values.T)
train_summary = pd.concat([train_summary_mean, train_summary_std], axis=1)

train_summary_mean = pd.DataFrame(mean_values.T)
train_summary_std = pd.DataFrame(std_values.T)

train_summary = pd.concat([train_summary_mean, train_summary_std], axis=1)
train_summary.insert(2, 'stand_out',pd.DataFrame(train_summary.sum(axis=1)))

In [ ]:
train_data.insert(5,'stand_out', train_summary[['stand_out']])
train_data

,대분류,중분류,소분류,브랜드,쇼핑몰,stand_out,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,4.866882,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,5.811748,0,0,0,0,...,2,0,2,0,2,2,1,0,0,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,11.908563,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,22.347660,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,64.043286,0,0,0,0,...,10,6,4,4,4,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,15.270999,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28890,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,79.001950,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28891,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,29.197243,0,0,0,0,...,2,4,1,0,0,0,4,0,2,0
28892,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,18.314348,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_out = train_data['stand_out'] < train_data.iloc[:, 6:].T

true_ratios = train_out.mean(axis=1)
true_ratios

#무슨 비율이더라 아 아웃라이어 비율

2022-01-01    0.093860
2022-01-02    0.099190
2022-01-03    0.114903
2022-01-04    0.126497
2022-01-05    0.130719
                ...   
2023-04-20    0.031944
2023-04-21    0.031287
2023-04-22    0.020662
2023-04-23    0.018481
2023-04-24    0.033398
Length: 479, dtype: float64

In [ ]:
threshold = 0.14
columns_to_delete = true_ratios[true_ratios > threshold].index




In [ ]:
train_data = train_data.drop(columns=columns_to_delete) #이상치 제거

In [ ]:
summary_brand = brand_keyword_cnt.drop(columns=['브랜드']).T.describe()
mean_brand = pd.DataFrame(summary_brand.loc['mean'])

In [ ]:
brand = brand_keyword_cnt[['브랜드']]
brand = pd.concat([brand, mean_brand], axis=1)

In [ ]:
brand

,브랜드,mean
0,B002-00001,0.702642
1,B002-00002,11.530036
2,B002-00003,0.799032
3,B002-00004,0.581572
4,B002-00005,1.906285
...,...,...
3817,B002-03818,0.219967
3818,B002-03819,0.000333
3819,B002-03820,0.001817
3820,B002-03821,13.752923


In [ ]:
brand[brand['mean'].isna()] # 아래 항목들은 원래 키워드 파일에서 아예 값이 없음

,브랜드,mean


In [ ]:
brand['mean'].fillna(0, inplace=True) # 0으로 채워준다.

In [ ]:
feature = train_data.iloc[:,:6]

In [ ]:
merged_df = pd.merge(feature,brand, how='inner',on='브랜드')
merged_df

,대분류,중분류,소분류,브랜드,쇼핑몰,stand_out,mean
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,4.866882,0.702642
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,5.811748,11.530036
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,11.908563,11.530036
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,22.347660,11.530036
4,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,64.043286,11.530036
...,...,...,...,...,...,...,...
28889,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,15.270999,0.147169
28890,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,79.001950,5.497862
28891,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,29.197243,5.497862
28892,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,18.314348,5.497862


In [ ]:
merged_df = merged_df.rename(columns={'mean':'b_k_mean'})

In [ ]:
train_data

,대분류,중분류,소분류,브랜드,쇼핑몰,stand_out,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,4.866882,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,5.811748,0,0,0,0,...,2,0,2,0,2,2,1,0,0,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,11.908563,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,22.347660,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,64.043286,0,0,0,0,...,10,6,4,4,4,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,15.270999,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28890,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,79.001950,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28891,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,29.197243,0,0,0,0,...,2,4,1,0,0,0,4,0,2,0
28892,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,18.314348,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data = pd.concat([merged_df, train_data.iloc[:,6:]], axis=1)

In [ ]:
train_zero_ratio = train_data.iloc[:, :] == 0
train_zero_ratio = pd.DataFrame(train_zero_ratio.apply(lambda row: row.value_counts(normalize=True).get(True, 0), axis=1))
train_zero_ratio.columns =['ratio']
train_data.insert(5, 'zero_ratio', train_zero_ratio)

In [ ]:
train_data.iloc[:,8:]

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,2,0,2,0,2,2,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,10,6,4,4,4,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28891,0,0,0,0,0,0,0,0,0,0,...,2,4,1,0,0,0,4,0,2,0
28892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 먼저 5번째 열부터의 데이터만 선택합니다.
data_subset = train_data.iloc[:, 8:]

# 각 행의 최댓값과 최솟값을 계산합니다.
max_values = data_subset.max(axis=1)
min_values = data_subset.min(axis=1)

# 분모가 0이 되는 경우를 처리하기 위해 조건을 적용합니다.
diff = max_values - min_values
mask = diff != 0

# 정규화 작업을 수행합니다.
train_data.loc[mask, data_subset.columns] = (data_subset[mask] - min_values[mask].values.reshape(-1, 1)) / diff[mask].values.reshape(-1, 1)
train_data.loc[~mask, data_subset.columns] = 0

# 결과를 사전에 저장합니다.
scale_max_dict = max_values.to_dict()
scale_min_dict = min_values.to_dict()

In [ ]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드', '쇼핑몰']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [ ]:
train_data

,대분류,중분류,소분류,브랜드,쇼핑몰,zero_ratio,stand_out,b_k_mean,2022-01-01,2022-01-02,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,1,6,37,0,0,0.724771,4.866882,0.702642,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,7,43,1,0,0.493119,5.811748,11.530036,0.0,0.0,...,0.142857,0.000000,0.142857,0.000000,0.142857,0.142857,0.071429,0.000000,0.000000,0.000000
2,2,7,43,1,0,0.405963,11.908563,11.530036,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2,7,43,1,0,0.422018,22.347660,11.530036,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2,7,43,1,9,0.839450,64.043286,11.530036,0.0,0.0,...,0.017606,0.010563,0.007042,0.007042,0.007042,0.000000,0.000000,0.000000,0.000000,0.014085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,2,7,43,2893,0,0.770642,15.270999,0.147169,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28890,2,7,41,2894,0,0.383028,79.001950,5.497862,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28891,2,7,43,2894,0,0.442661,29.197243,5.497862,0.0,0.0,...,0.008130,0.016260,0.004065,0.000000,0.000000,0.000000,0.016260,0.000000,0.008130,0.000000
28892,2,7,43,2894,0,0.405963,18.314348,5.497862,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
train_data = train_data.drop(columns=['stand_out'])

In [ ]:
train_data

,대분류,중분류,소분류,브랜드,쇼핑몰,zero_ratio,b_k_mean,2022-01-01,2022-01-02,2022-01-03,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,1,6,37,0,0,0.724771,0.702642,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,7,43,1,0,0.493119,11.530036,0.0,0.0,0.0,...,0.142857,0.000000,0.142857,0.000000,0.142857,0.142857,0.071429,0.000000,0.000000,0.000000
2,2,7,43,1,0,0.405963,11.530036,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2,7,43,1,0,0.422018,11.530036,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2,7,43,1,9,0.839450,11.530036,0.0,0.0,0.0,...,0.017606,0.010563,0.007042,0.007042,0.007042,0.000000,0.000000,0.000000,0.000000,0.014085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,2,7,43,2893,0,0.770642,0.147169,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28890,2,7,41,2894,0,0.383028,5.497862,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28891,2,7,43,2894,0,0.442661,5.497862,0.0,0.0,0.0,...,0.008130,0.016260,0.004065,0.000000,0.000000,0.000000,0.016260,0.000000,0.008130,0.000000
28892,2,7,43,2894,0,0.405963,5.497862,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE'], is_inference=False):
        self.data = data.values # convert DataFrame to numpy array
        self.train_size = train_size
        self.predict_size = predict_size
        self.window_size = self.train_size + self.predict_size
        self.is_inference = is_inference

    def __len__(self):
        if self.is_inference:
            return len(self.data)
        else:
            return self.data.shape[0] * (self.data.shape[1] - self.window_size - 6)

    def __getitem__(self, idx):
        if self.is_inference:
            # 추론 시
            encode_info = self.data[idx, :7]
            window = self.data[idx, -self.train_size:]
            input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window))
            return input_data
        else:
            # 학습 시
            row = idx // (self.data.shape[1] - self.window_size - 6)
            col = idx % (self.data.shape[1] - self.window_size - 6)
            encode_info = self.data[row, :7]
            sales_data = self.data[row, 7:]
            window = sales_data[col : col + self.window_size]
            input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window[:self.train_size]))
            target_data = window[self.train_size:]
            return input_data, target_data

In [ ]:
# CustomDataset 인스턴스 생성
dataset = CustomDataset(train_data)

# 전체 데이터셋의 크기
total_size = len(dataset)

# 분리할 데이터셋의 크기 계산
train_size = int(total_size * 0.8)
val_size = total_size - train_size

# random_split 함수를 사용해 데이터셋 분리
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoader 인스턴스 생성
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, input_size=8, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 6)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden)

        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

In [ ]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())
    return np.mean(val_loss)

In [ ]:
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/1801 [00:00<?, ?it/s]

In [ ]:
test_dataset = CustomDataset(data=train_data, is_inference=True)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [ ]:
def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.float().to(device)

            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')
submit.head()

In [ ]:
submit.iloc[:,1:] = pred
submit.head()

In [ ]:
submit.to_csv('/content/drive/MyDrive/submission_v2.csv', index=False)